<a href="https://colab.research.google.com/github/nycolerodrigues/case-automec/blob/main/Case_Automec_Solutions_Nycole_Costa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

######NYCOLE MARIA COSTA RODRIGUES - CASE PARA VAGA DE ESTÁGIO NA PEERS

######**`CASE AUTOMEC SOLUTIONS`**

###### Informações importantes:

###### Algumas métricas reutilizam resultados de métricas anteriores.
###### Portanto, ao testar, execute sempre na ordem (1 → 10) para evitar erros.
###### Obrigada :)


###### 1º passo: Importar os arquivos CSV do repositório no GitHub
###### Antes de iniciar qualquer execução, rode esta célula para carregar os arquivos CSV do repositório.


In [ ]:
'''Preferi transformar cada aba de dados da planilha para um CSV para me ajudar na organização dos dados
e hospedei no meu repositório do GitHub para que o acesso seja mais simples.
'''
import pandas as pd
# URLs dos arquivos
url_vendas = 'https://raw.githubusercontent.com/nycolerodrigues/case-automec/refs/heads/main/Transa%C3%A7%C3%B5es_Vendas.csv'
url_clientes = 'https://raw.githubusercontent.com/nycolerodrigues/case-automec/refs/heads/main/Cadastro_Clientes.csv'
url_produtos = 'https://raw.githubusercontent.com/nycolerodrigues/case-automec/refs/heads/main/Cadastro_Produtos.csv'
url_estoque = 'https://raw.githubusercontent.com/nycolerodrigues/case-automec/refs/heads/main/Cadastro_de_Estoque.csv'
url_fornecedores ='https://raw.githubusercontent.com/nycolerodrigues/case-automec/refs/heads/main/Cadastro_Fornecedores.csv'

try:
    # Lendo os arquivos
    transacoes_vendas = pd.read_csv(url_vendas, encoding='utf-8')
    cadastro_clientes = pd.read_csv(url_clientes, encoding='utf-8')
    cadastro_produtos = pd.read_csv(url_produtos, encoding='utf-8')
    cadastro_estoque = pd.read_csv(url_estoque, encoding='utf-8')
    cadastro_fornecedores = pd.read_csv(url_fornecedores, encoding='utf-8')
    print("Carregou com sucesso! Eba :)")

except Exception as e:
    print(f"Não foi possível ler os arquivos da URL. Paia:( \n")
    print(f"Erro: {e}")

######Visualização inicial dos dados (não é necessário executar)


In [ ]:
'''
--- Verificação inicial dos dados ---
Fiz uma verificação inicial nos dados para confirmar se a importação dos arquivos ocorreu corretamente, deixei comentado po questões de documetação

--- Informações sobre as tabelas ---
info_vendas = transacoes_vendas.info()
info_clientes = cadastro_clientes.info()
info_produtos = cadastro_produtos.info()
info_estoque = cadastro_estoque.info()
info_fornecedores = cadastro_fornecedores.info()

--- Visualizando as tabelas ---
display(transacoes_vendas.head())
display(cadastro_clientes.head())
display(cadastro_produtos.head())
display(cadastro_estoque.head())
display(cadastro_fornecedores.head())
'''

######**Resultado das métricas**


###### Todos os códigos das métricas estão agrupados dentro dos seus respectivos enunciados.
###### Para visualizar o código e os resultados, basta expandir a célula clicando na setinha ( > ) antes do enunciado.


######1) Calcule o valor total de venda dos produtos por categoria. Utilize as tabelas CADASTRO_PRODUTOS e TRANSACOES_VENDAS.

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO: Identificar qual categoria de produtos apresenta o maior e menor valor total de vendas

#1. Construção das tabelas
#Faz o merge entre as tabelas CADASTRO_PRODUTOS e TRANSACOES_VENDAS conectando a venda ao seu respectivo produto e categoria
vendas_com_produtos = pd.merge(
    transacoes_vendas,
    cadastro_produtos,
    on = 'ID PRODUTO',
    how='inner'
)

#2. Agrupamento e cálculo do valor total de vendas por categoria
# Agrupa os produtos por categoria e soma o valor total de vendas (VALOR NOTA).
# Em seguida, renomeia a coluna e ordena da maior para a menor categoria em valor vendido.

total_vendas_por_categoria =(
    vendas_com_produtos
    .groupby('CATEGORIA', as_index=False)['VALOR NOTA']
    .sum()
    .rename(columns={'VALOR NOTA': 'VALOR TOTAL'})
    .sort_values(by='VALOR TOTAL', ascending=False)
)
# Calcula o total geral de vendas
total_geral = total_vendas_por_categoria['VALOR TOTAL'].sum()

# Adiciona uma coluna de percentual de participação
total_vendas_por_categoria['% PARTICIPAÇÃO'] = (
    total_vendas_por_categoria['VALOR TOTAL'] / total_geral.round(4)
)

#3. Resultados
# Exibe o valor monetário e a porcentagem que cada categoria representa no faturamento
# Cálculo do Faturamento Total
faturamento_total = transacoes_vendas['VALOR NOTA'].sum()

# Formatação para Padrão BR
faturamento_formatado = f"R$ {faturamento_total:,.2f}".replace(',', 'v').replace('.', ',').replace('v', '.')

print(f"💰 Faturamento total da Automec Solutions: {faturamento_formatado}\n")
print("Valor total de vendas e % de participação por categoria:\n")
display(
    total_vendas_por_categoria.style
    .format({
        'VALOR TOTAL': lambda x: f"R$ {x:,.2f}"
        .replace(',', 'v')
        .replace('.', ',')
        .replace('v', '.'),
        '% PARTICIPAÇÃO': lambda x: f"{x*100:,.2f}%"
        .replace(',', 'v')
        .replace('.', ',')
        .replace('v', '.')
    })
    .hide(axis='index')
)


######2) Calcule a margem dos produtos subtraindo o valor unitario pelo valor de venda. Utilize as tabelas CADASTRO_ESTOQUE e TRANSACOES_VENDAS.

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO: Calcular a margem de cada venda individual, identificando o fornecedor e o custo específico do lote.
# Isso permite analisar a variação de rentabilidade e identificar se o prejuízo vem da precificação ou do custo de compra.

# 1. Construção das tabelas e preparação dos dados

# Limpeza da coluna VALOR ESTOQUE (removendo R$ e formatando para float)
# Calculamos o CUSTO UNITARIO real de cada lote (Valor Total / Quantidade)
cadastro_estoque_limpo = cadastro_estoque.copy()

if pd.api.types.is_string_dtype(cadastro_estoque_limpo['VALOR ESTOQUE']):
    cadastro_estoque_limpo['VALOR ESTOQUE'] = (
        cadastro_estoque_limpo['VALOR ESTOQUE']
        .astype(str)
        .str.replace('R$', '', regex=False)
        .str.replace(r'\.', '', regex=True)
        .str.replace(',', '.', regex=False)
        .str.strip()
        .astype(float)
    )

cadastro_estoque_limpo['CUSTO UNITARIO'] = (
    cadastro_estoque_limpo['VALOR ESTOQUE'] / cadastro_estoque_limpo['QTD ESTOQUE']
)

# Traz o NOME DO FORNECEDOR para a tabela de estoque (para identificar a origem do custo)
estoque_com_fornecedor = pd.merge(
    cadastro_estoque_limpo,
    cadastro_fornecedores[['ID FORNECEDOR', 'NOME FORNECEDOR']],
    on='ID FORNECEDOR',
    how='left'
)

# Prepara a tabela de custos completa para conectar com produtos
dados_custo_completo = estoque_com_fornecedor[['ID ESTOQUE', 'CUSTO UNITARIO', 'NOME FORNECEDOR']]

# Conecta Produtos -> Custos/Fornecedores -> Vendas
# Isso garante que cada venda tenha o custo exato do lote vendido
produtos_com_custo = pd.merge(
    cadastro_produtos,
    dados_custo_completo,
    on='ID ESTOQUE',
    how='left'
)

vendas_com_margem = pd.merge(
    transacoes_vendas,
    produtos_com_custo,
    on='ID PRODUTO',
    how='left'
)

# Garante que os tipos de dados estejam corretos para cálculos matemáticos e datas
vendas_com_margem['VALOR ITEM'] = pd.to_numeric(vendas_com_margem['VALOR ITEM'], errors='coerce')
vendas_com_margem['DATA NOTA'] = pd.to_datetime(vendas_com_margem['DATA NOTA'])


# 2. Cálculos de margem

# Calcula a Margem em Reais
vendas_com_margem['MARGEM_UNITARIA'] = (
    vendas_com_margem['VALOR ITEM'] - vendas_com_margem['CUSTO UNITARIO']
)

# Calcula a Margem Percentual
vendas_com_margem['MARGEM_PERCENTUAL'] = (
    (vendas_com_margem['MARGEM_UNITARIA'] / vendas_com_margem['CUSTO UNITARIO']) * 100
)

# Cria uma coluna de Status para facilitar a visualização rápida (Lucro vs Prejuízo)
vendas_com_margem['STATUS'] = vendas_com_margem['MARGEM_UNITARIA'].apply(
    lambda x: 'LUCRO' if x > 0 else 'PREJUIZO' if x < 0 else 'ZERADO'
)


# 3. Resultados

# Seleciona e renomeia as colunas para a tabela final
resultado_detalhado = vendas_com_margem[[
    'ID NOTA',
    'DATA NOTA',
    'NOME PRODUTO',
    'NOME FORNECEDOR',
    'VALOR ITEM',
    'CUSTO UNITARIO',
    'MARGEM_UNITARIA',
    'MARGEM_PERCENTUAL',
    'STATUS'
]].copy()

resultado_detalhado = resultado_detalhado.rename(columns={
    'VALOR ITEM': 'Valor Venda',
    'CUSTO UNITARIO': 'Custo Compra',
    'MARGEM_UNITARIA': 'Lucro/Prejuízo',
    'MARGEM_PERCENTUAL': 'Margem %',
    'NOME FORNECEDOR': 'Fornecedor',
    'STATUS': 'Status'
})

# Ordena por Produto e Data para permitir a análise da evolução histórica de preços e margens
resultado_detalhado = resultado_detalhado.sort_values(
    by=['NOME PRODUTO', 'DATA NOTA'],
    ascending=[True, True]
)

# Formatação para o padrão brasileiro (R$ X.XXX,XX e X,X%)
formatacao_brl = lambda x: f"R$ {x:,.2f}".replace(',', 'v').replace('.', ',').replace('v', '.')
formatacao_perc = lambda x: f"{x:,.1f}%".replace(',', 'v').replace('.', ',').replace('v', '.')

# Ajusta a data para exibição (dd/mm/aaaa)
resultado_detalhado['DATA NOTA'] = resultado_detalhado['DATA NOTA'].dt.strftime('%d/%m/%Y')

print("Margem de lucro por venda e fornecedor:")

# Exibe a tabela com formatação condicional:
# Vermelho para valores negativos (Prejuízo) e Verde para positivos (Lucro)
display(
    resultado_detalhado.style
    .format({
        'Valor Venda': formatacao_brl,
        'Custo Compra': formatacao_brl,
        'Lucro/Prejuízo': formatacao_brl,
        'Margem %': formatacao_perc
    })
    .map(lambda x: 'color: red' if isinstance(x, (float, int)) and x < 0 else 'color: green' if isinstance(x, (float, int)) and x > 0 else '',
         subset=['Lucro/Prejuízo', 'Margem %'])
    .hide(axis='index')
)

######3) Calcule um ranking de clientes por quantidade de produtos comprados por mês. Utilize as tabelas CADASTRO_CLIENTES e TRANSACOES_VENDAS.

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO: Analisar o comportamento de compra detalhado por cliente, identificando
# quais produtos e categorias cada cliente comprou ao longo dos meses

# 1. Construção das tabelas e Preparação dos dados

# Cria uma cópia da transacoes_vendas para não alterar diretamente o CSV
vendas_com_data = transacoes_vendas.copy()

# Converte a 'DATA NOTA' para o formato datetime e cria 'Ano-Mes' para ordenação
vendas_com_data['DATA NOTA'] = pd.to_datetime(vendas_com_data['DATA NOTA'])
vendas_com_data['Ano-Mes'] = vendas_com_data['DATA NOTA'].dt.to_period('M')

# Merge 1: Conecta as Vendas aos Clientes
vendas_com_clientes = pd.merge(vendas_com_data, cadastro_clientes, on='ID CLIENTE')

# Merge 2: Conecta o resultado aos Produtos
# Isso traz o NOME PRODUTO e a CATEGORIA para a tabela de vendas
vendas_completa = pd.merge(vendas_com_clientes, cadastro_produtos, on='ID PRODUTO')

# Agrupa por Mês, Cliente, Categoria E Produto para ver o detalhe de cada compra
ranking_clientes_detalhado = vendas_completa.groupby(
    ['Ano-Mes', 'NOME CLIENTE', 'CATEGORIA', 'NOME PRODUTO']
)['QTD ITEM'].sum().reset_index()


# 2. Definições de Funções de Estilo
# Dicionário de tradução dos meses para o português
mapa_meses = {
    'January': 'Janeiro', 'February': 'Fevereiro', 'March': 'Março',
    'April': 'Abril', 'May': 'Maio', 'June': 'Junho',
    'July': 'Julho', 'August': 'Agosto', 'September': 'Setembro',
    'October': 'Outubro', 'November': 'Novembro', 'December': 'Dezembro'
}
# Dicionário de cores para a célula do Mês
mapa_cores_meses = {
    'Janeiro': '#B0E0E6',   # PowderBlue (Azul)
    'Fevereiro': '#F08080',   # LightCoral (Vermelho/Rosa)
    'Março': '#98FB98',   # PaleGreen (Verde)
    'Abril': '#F0E68C',   # Khaki (Amarelo)
    'Maio': '#DDA0DD',   # Plum (Roxo/Magenta)
    'Junho': '#B0C4DE',   # LightSteelBlue (Azul Aço)
    'Julho': '#F4A460',   # SandyBrown (Laranja)
    'Agosto': '#AFEEEE',   # PaleTurquoise (Turquesa)
    'Setembro': '#6495ED',   # CornflowerBlue (Azul Médio)
    'Outubro': '#FFA07A',   # LightSalmon (Salmão)
    'Novembro': '#D3D3D3',   # LightGray (Cinza)
    'Dezembro': '#8FBC8F'    # DarkSeaGreen (Verde Musgo)
}

# Aplica a cor na CÉLULA do Mês (baseado no dicionário)
def estilo_celula_mes(nome_mes):
    cor_fundo = mapa_cores_meses.get(nome_mes.strip(), 'white')
    # Retorna o CSS com fundo colorido e texto escuro
    return f'background-color: {cor_fundo}; color: #111; font-weight: 500; border-bottom: 1px solid #aaa'

# Adiciona uma borda dourada QUANDO O CLIENTE MUDA (O SEPARADOR)
def highlight_client_change(df_to_style):
    # Cria um DataFrame de estilos vazio
    styler_df = pd.DataFrame('', index=df_to_style.index, columns=df_to_style.columns)
    # Cria uma máscara (True) onde o nome é diferente da linha anterior
    mask = df_to_style['NOME CLIENTE'] != df_to_style['NOME CLIENTE'].shift()
    # Aplica a borda de 3px dourada onde a máscara é True
    styler_df.loc[mask, :] = 'border-top: 3px solid #FFD700;'
    return styler_df


# 3. Refinamento e Formatação dos Resultados
# Renomeia a coluna de quantidade para 'QTD'
ranking_clientes_detalhado = ranking_clientes_detalhado.rename(columns={'QTD ITEM': 'QTD'})

# Ordena por Cliente (A-Z) e depois por Mês (Cronológico)
ranking_clientes_detalhado = ranking_clientes_detalhado.sort_values(
    by=['NOME CLIENTE', 'Ano-Mes'],
    ascending=[True, True]
)

# Cria as colunas 'Ano' e 'Mês' (Nome em Inglês)
ranking_clientes_detalhado['Ano'] = ranking_clientes_detalhado['Ano-Mes'].dt.year
ranking_clientes_detalhado['Mês'] = ranking_clientes_detalhado['Ano-Mes'].dt.to_timestamp().dt.strftime('%B')

# Traduz a coluna 'Mês' para Português usando o 'mapa_meses'
ranking_clientes_detalhado['Mês'] = ranking_clientes_detalhado['Mês'].replace(mapa_meses, regex=True)

# Separa os DataFrames por ano
ranking_2023 = ranking_clientes_detalhado[ranking_clientes_detalhado['Ano'] == 2023].copy()
ranking_2024 = ranking_clientes_detalhado[ranking_clientes_detalhado['Ano'] == 2024].copy()

# Seleciona as colunas finais para exibição
colunas_finais = ['Mês', 'NOME CLIENTE', 'CATEGORIA', 'NOME PRODUTO', 'QTD']
ranking_2023_final = ranking_2023[colunas_finais]
ranking_2024_final = ranking_2024[colunas_finais]

# 4. Resultados
# Define o estilo do cabeçalho (Preto com texto branco)
estilo_cabecalho = [
    {'selector': 'th', 'props': [('background-color', '#333'), ('color', 'white'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('border-bottom', '1px solid #aaa')]}
]

# Exibe a tabela de 2024
print("\n Historico de compras - 2024: ")
display(
    ranking_2024_final.style
    .map(estilo_celula_mes, subset=['Mês'])
    .apply(highlight_client_change, axis=None)
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)

# Exibe a tabela de 2023
print("\n 2. Histórico de compras - 2023: ")
display(
    ranking_2023_final.style
    .map(estilo_celula_mes, subset=['Mês'])
    .apply(highlight_client_change, axis=None)
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)

######4) Calcule um ranking de fornecedores por quantidade de estoque disponivel por mês. Utilize as tabelas CADASTRO_FORNECEDORES e CADASTRO_ESTOQUE.

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO: Analisar a Quantidade (QTD) do estoque por fornecedor

# 1. Construção das tabelas e preparação dos dados

# Cria uma cópia para trabalhar com as datas
estoque_com_data = cadastro_estoque_limpo.copy()

# Converte a 'DATA ESTOQUE' para o formato datetime e cria 'Ano-Mes'
estoque_com_data['DATA ESTOQUE'] = pd.to_datetime(estoque_com_data['DATA ESTOQUE'])
estoque_com_data['Ano-Mes'] = estoque_com_data['DATA ESTOQUE'].dt.to_period('M')

# Junta com a tabela de fornecedores
estoque_com_fornecedores = pd.merge(
    estoque_com_data,
    cadastro_fornecedores,
    on='ID FORNECEDOR'
)

# 2. Cálculos
# Agrupa por Mês e Fornecedor, somando QTD e VALOR DO ESTOQUE
ranking_fornecedores_mes = estoque_com_fornecedores.groupby(
    ['Ano-Mes', 'NOME FORNECEDOR'], as_index=False
).agg(
    QTD_TOTAL=('QTD ESTOQUE', 'sum'),
    VALOR_TOTAL=('VALOR ESTOQUE', 'sum')
)

# Ordena pelo Mês e, dentro do mês, pela MAIOR QUANTIDADE (QTD)
ranking_fornecedores_mes = ranking_fornecedores_mes.sort_values(
    by=['Ano-Mes', 'QTD_TOTAL'],
    ascending=[True, False]
)

# 3. Definições de Funções de Estilo
mapa_meses = {
    'January': 'Janeiro', 'February': 'Fevereiro', 'March': 'Março',
    'April': 'Abril', 'May': 'Maio', 'June': 'Junho',
    'July': 'Julho', 'August': 'Agosto', 'September': 'Setembro',
    'October': 'Outubro', 'November': 'Novembro', 'December': 'Dezembro'
}

mapa_cores_meses = {
    'Janeiro': '#B0E0E6', 'Fevereiro': '#F08080', 'Março': '#98FB98',
    'Abril': '#F0E68C', 'Maio': '#DDA0DD', 'Junho': '#B0C4DE',
    'Julho': '#F4A460', 'Agosto': '#AFEEEE', 'Setembro': '#6495ED',
    'Outubro': '#FFA07A', 'Novembro': '#D3D3D3', 'Dezembro': '#8FBC8F'
}

def estilo_celula_mes(nome_mes):
    cor_fundo = mapa_cores_meses.get(nome_mes.strip(), 'white')
    return f'background-color: {cor_fundo}; color: #111; font-weight: 500; border-bottom: 1px solid #aaa'

def highlight_month_change(df_to_style):
    styler_df = pd.DataFrame('', index=df_to_style.index, columns=df_to_style.columns)
    mask = df_to_style['Mês'] != df_to_style['Mês'].shift()
    styler_df.loc[mask, :] = 'border-top: 3px solid #FFD700;'
    return styler_df

# 4. Refinamento e formatação dos resultados
ranking_fornecedores_mes['Ano'] = ranking_fornecedores_mes['Ano-Mes'].dt.year
ranking_fornecedores_mes['Mês'] = ranking_fornecedores_mes['Ano-Mes'].dt.to_timestamp().dt.strftime('%B')
ranking_fornecedores_mes['Mês'] = ranking_fornecedores_mes['Mês'].replace(mapa_meses, regex=True)

ranking_geral_2022 = ranking_fornecedores_mes[ranking_fornecedores_mes['Ano'] == 2022].copy()
ranking_geral_2023 = ranking_fornecedores_mes[ranking_fornecedores_mes['Ano'] == 2023].copy()

colunas_finais = ['Mês', 'NOME FORNECEDOR', 'QTD_TOTAL', 'VALOR_TOTAL']
ranking_geral_2022_final = ranking_geral_2022[colunas_finais]
ranking_geral_2023_final = ranking_geral_2023[colunas_finais]

# 5. Resultados
formatacao_brl = lambda x: f"R$ {x:,.2f}".replace(',', 'v').replace('.', ',').replace('v', '.')
estilo_cabecalho = [
    {'selector': 'th', 'props': [('background-color', '#333'), ('color', 'white'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('border-bottom', '1px solid #aaa')]}
]

print("\n Fornecedores e quantidade de estoque - 2022:\n ")
display(
    ranking_geral_2022_final.style
    .map(estilo_celula_mes, subset=['Mês'])
    .apply(highlight_month_change, axis=None)
    .format({'VALOR_TOTAL': formatacao_brl})
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)
print("\n Fornecedores e quantidade de estoque - 2023:\n")
display(
    ranking_geral_2023_final.style
    .map(estilo_celula_mes, subset=['Mês'])
    .apply(highlight_month_change, axis=None)
    .format({'VALOR_TOTAL': formatacao_brl})
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)

######5) Calcule um ranking de produtos por quantidade de venda por mês. Utilize a tabela TRANSACOES_VENDAS.
######6) Calcule um ranking de produtos por valor de venda por mês. Utilize a tabela TRANSACOES_VENDAS.

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO: Criar dois rankings de produtos (Métrica 5: Quantidade; Métrica 6: Valor)

#Resolvi criar o código para as duas metricas juntos pois eles são bem parecidos

# 1. Construção das tabelas e preparação dos dados

# Merge 1: Vendas + Produtos
vendas_com_produtos = pd.merge(
    transacoes_vendas,
    cadastro_produtos,
    on='ID PRODUTO',
    how='inner'
)
# Converte a 'DATA NOTA' para datetime e cria 'Ano-Mes' para agrupar e ordenar
vendas_com_produtos['DATA NOTA'] = pd.to_datetime(vendas_com_produtos['DATA NOTA'])
vendas_com_produtos['Ano-Mes'] = vendas_com_produtos['DATA NOTA'].dt.to_period('M')

# 2. Agrupamento e Cálculo
# Agrupa por Mês, Categoria e Produto, somando a QTD (Métrica 5) e o VALOR (Métrica 6)

ranking_produtos_mes = vendas_com_produtos.groupby(
    ['Ano-Mes', 'CATEGORIA', 'NOME PRODUTO'], as_index=False
).agg(
    QTD_TOTAL=('QTD ITEM', 'sum'),
    VALOR_TOTAL=('VALOR NOTA', 'sum')
).reset_index(drop=True)

# 3. Refinamento e Formatação
# Cria as colunas 'Ano' e 'Mês' (Nome em Inglês)
ranking_produtos_mes['Ano'] = ranking_produtos_mes['Ano-Mes'].dt.year
ranking_produtos_mes['Mês'] = ranking_produtos_mes['Ano-Mes'].dt.to_timestamp().dt.strftime('%B')

# Traduz a coluna 'Mês' para Português (reutilizando 'mapa_meses' da Métrica 3)
ranking_produtos_mes['Mês'] = ranking_produtos_mes['Mês'].replace(mapa_meses, regex=True)

# Define as colunas que queremos mostrar
colunas_finais = ['Ano', 'Mês', 'CATEGORIA', 'NOME PRODUTO', 'QTD_TOTAL', 'VALOR_TOTAL']


# 4. Resultados
# (Reutiliza as funções 'estilo_celula_mes', 'highlight_month_change',
# 'formatacao_brl' e 'estilo_cabecalho' definidas anteriormente)

# --- SAÍDA DA MÉTRICA 5 (Ranking por QUANTIDADE) ---
print("\n MÉTRICA 5: Ranking de Produtos por QUANTIDADE:\n")

# Ordena pela data (Ano-Mes) e depois pela QTD_Total (maior para menor)
ranking_qtd_final = ranking_produtos_mes.sort_values(
    by=['Ano-Mes', 'QTD_TOTAL'],
    ascending=[True, False]
)
display(
    ranking_qtd_final[colunas_finais].head(10).style
    .apply(highlight_month_change, axis=None)
    .format({'VALOR_TOTAL': formatacao_brl})
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)

# --- SAÍDA DA MÉTRICA 6 (Ranking por VALOR) ---
print("\n\n MÉTRICA 6: Ranking de Produtos por valor de venda: ")

# Ordena pela data (Ano-Mes) e depois pelo VALOR_Total_R (maior para menor)
ranking_val_final = ranking_produtos_mes.sort_values(
    by=['Ano-Mes', 'VALOR_TOTAL'],
    ascending=[True, False]
)

display(
    ranking_val_final[colunas_finais].head(10).style
    .apply(highlight_month_change, axis=None)
    .format({'VALOR_TOTAL': formatacao_brl})
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)

######7) Calcule a média de valor de venda por categoria de produto por mês. Utiliza as tabelas CADASTRO_PRODUTOS e TRANSACOES_VENDAS.

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO:Calcular o "Ticket Médio" (Valor Médio da Venda) por categoria e por mês.

# 1. Construção das tabelas e preparação dos dados
# Merge 1: Vendas + Produtos
vendas_com_produtos = pd.merge(
    transacoes_vendas,
    cadastro_produtos,
    on='ID PRODUTO',
    how='inner'
)
# Converte a 'DATA NOTA' para datetime e cria 'Ano-Mes' para agrupar
vendas_com_produtos['DATA NOTA'] = pd.to_datetime(vendas_com_produtos['DATA NOTA'])
vendas_com_produtos['Ano-Mes'] = vendas_com_produtos['DATA NOTA'].dt.to_period('M')

# 2. Agrupamento e Cálculo
# Agrupa por Mês e Categoria, calculando a MÉDIA do VALOR NOTA
media_categoria_mes = vendas_com_produtos.groupby(
    ['Ano-Mes', 'CATEGORIA'], as_index=False
)['VALOR NOTA'].mean()

# Renomeia a coluna
media_categoria_mes = media_categoria_mes.rename(columns={'VALOR NOTA': 'Ticket Médio (R$)'})


# 3. Refinamento e Formatação
# Cria as colunas 'Ano' e 'Mês' (Nome em Inglês)
media_categoria_mes['Ano'] = media_categoria_mes['Ano-Mes'].dt.year
media_categoria_mes['Mês'] = media_categoria_mes['Ano-Mes'].dt.to_timestamp().dt.strftime('%B')

# Traduz a coluna 'Mês' para Português (reutilizando 'mapa_meses' da Métrica 3)
media_categoria_mes['Mês'] = media_categoria_mes['Mês'].replace(mapa_meses, regex=True)

# Ordena por data (Ano-Mes) e pelo Ticket Médio (maior para menor)
media_categoria_mes = media_categoria_mes.sort_values(
    by=['Ano-Mes', 'Ticket Médio (R$)'],
    ascending=[True, False]
)
# Seleciona as colunas finais
colunas_finais = ['Ano', 'Mês', 'CATEGORIA', 'Ticket Médio (R$)']
ranking_final_media = media_categoria_mes[colunas_finais]


# 4. Resultados
# (Reutiliza as funções 'estilo_celula_mes', 'highlight_month_change',
# 'formatacao_brl' e 'estilo_cabecalho' definidas anteriormente)

print("\nTicket Médio por categoria:\n")
display(
    ranking_final_media.style
    .apply(highlight_month_change, axis=None)
    .format({'Ticket Médio (R$)': formatacao_brl})
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)
#--------------------------------------------------------------------------------------------------------------------#

######8) Calcule um ranking de margem de lucro por categoria

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO: Calcular um ranking de margem de lucro por categoria.

# 1. Preparação dos dados
# Reutilizei a tabela 'vendas_com_margem' da Métrica 2.
# Ela já contém 'VALOR ITEM', 'CUSTO UNITARIO' e 'CATEGORIA'.

# Calcula o Lucro/Prejuízo da TRANSAÇÃO (Margem unitária * Quantidade)

vendas_com_margem['LUCRO_PREJUIZO_TOTAL'] = (
    vendas_com_margem['MARGEM_UNITARIA'] * vendas_com_margem['QTD ITEM']
)

# 2. Agrupamento e Cálculo
# Agrupa por CATEGORIA e soma o Faturamento Total (VALOR NOTA)
# e o Lucro/Prejuízo Total (LUCRO_PREJUIZO_TOTAL)
ranking_categoria = vendas_com_margem.groupby(
    'CATEGORIA', as_index=False
).agg(
    FATURAMENTO_TOTAL=('VALOR NOTA', 'sum'),
    LUCRO_TOTAL=('LUCRO_PREJUIZO_TOTAL', 'sum')
)

# 3. Refinamento e Formatação
# Calcula a Margem Percentual Final (Lucro Total / Faturamento Total)

ranking_categoria['MARGEM %'] = (
    ranking_categoria['LUCRO_TOTAL'] / ranking_categoria['FATURAMENTO_TOTAL']
).replace([float('inf'), float('-inf')], 0)

# Ordena pelo maior prejuízo
ranking_categoria_final = ranking_categoria.sort_values(by='LUCRO_TOTAL', ascending=True)

# Renomeia as colunas para a exibição final
ranking_categoria_final = ranking_categoria_final.rename(columns={
    'FATURAMENTO_TOTAL': 'Faturamento Total (R$)',
    'LUCRO_TOTAL': 'Lucro/Prejuízo Total (R$)'
})

colunas_finais = ['CATEGORIA', 'Faturamento Total (R$)', 'Lucro/Prejuízo Total (R$)', 'MARGEM %']


# 4. Resultados
# (Reutiliza as funções 'formatacao_brl' e 'estilo_cabecalho')
# (Reutiliza a formatação condicional de cor da Métrica 2)

print("\nRanking  do lucro/prejuízo por categoria:\n")

display(
    ranking_categoria_final[colunas_finais].style
    .format({
        'Faturamento Total (R$)': formatacao_brl,
        'Lucro/Prejuízo Total (R$)': formatacao_brl,
        'MARGEM %': '{:.2%}'
    })
    .map(lambda x: 'color: red' if isinstance(x, (float, int)) and x < 0 else 'color: green' if isinstance(x, (float, int)) and x > 0 else '',
         subset=['Lucro/Prejuízo Total (R$)', 'MARGEM %'])
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)
#--------------------------------------------------------------------------------------------------------------------#

######9) Liste produtos comprados por clientes

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO:Listar o "catálogo" de produtos que cada cliente já comprou.

# 1. Construção das tabelas
# Precisamos juntar 3 tabelas para conectar o Cliente ao Produto
vendas_com_clientes = pd.merge(
    transacoes_vendas,
    cadastro_clientes,
    on='ID CLIENTE'
)
vendas_clientes_produtos = pd.merge(
    vendas_com_clientes,
    cadastro_produtos,
    on='ID PRODUTO'
)

# 2. Refinamento e Formatação
# Seleciona apenas as colunas de interesse
lista_compras = vendas_clientes_produtos[['NOME CLIENTE', 'CATEGORIA', 'NOME PRODUTO']]

# Ordena por Cliente
lista_compras_unicas = lista_compras.drop_duplicates().sort_values(by='NOME CLIENTE')


# 3. Resultados
print("\n Catálogo de produtos comprados por cliente \n")

# (Reutiliza a função 'highlight_client_change' e 'estilo_cabecalho' da Métrica 3)
# O separador dourado aqui agrupa visualmente o "carrinho" de cada cliente.
display(
    lista_compras_unicas.style
    .apply(highlight_client_change, axis=None)
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)
#--------------------------------------------------------------------------------------------------------------------#

######10) Ranking de produtos por quantidade de estoque

In [ ]:
#--------------------------------------------------------------------------------------------------------------------#
# OBJETIVO:Calcular um ranking de produtos por quantidade de estoque

# 1. Construção das tabelas
# Junta Produtos + Estoque
produtos_com_estoque = pd.merge(
    cadastro_produtos,
    cadastro_estoque,
    on='ID ESTOQUE',
    how='inner'
)
# 2. Agrupamento e Cálculo
# Agrupa por produto e SOMA o estoque
ranking_estoque = produtos_com_estoque.groupby(
    ['CATEGORIA', 'NOME PRODUTO'], as_index=False
)['QTD ESTOQUE'].sum()

# 3. Resultados
ranking_estoque_final = ranking_estoque.sort_values(by='QTD ESTOQUE', ascending=False)
print("\n Ranking de produtos por quantidade de estoque: \n")
display(
    ranking_estoque_final.style
    .hide(axis='index')
    .set_table_styles(estilo_cabecalho)
)
#--------------------------------------------------------------------------------------------------------------------#

##### CONCLUSÃO E PRINCIPAIS INSIGHTS

Com base nas 10 métricas calculadas, a análise revela um cenário **crítico** para a Automec Solutions, mas com caminhos claros para a recuperação. A empresa enfrenta um problema estrutural de rentabilidade negativa, mas possui uma base de clientes fiéis e oportunidades de melhoria evidentes.

A resposta direta para a pergunta do case ("*Quais produtos e categorias geram maior e menor rentabilidade, e de que forma podemos aprimorar as estratégias de fornecimento e precificação para maximizar os lucros?*") é:

### 1. Diagnóstico: Prejuízo Generalizado (Métricas 2 e 8)

O *insight* mais importante é que a empresa está operando com **prejuízo em todas as 7 categorias** de produtos.

* **O Problema:** A Métrica 2 (detalhada) provou que a empresa está vendendo produtos consistentemente com o **Preço de Venda (Valor Item) abaixo do Custo de Compra (Custo Unitário)**.
* **A Causa:** A Métrica 8 confirmou que esse não é um problema pontual, mas sim sistêmico, levando a um **Lucro/Prejuízo Total negativo** em toda a empresa.

### 2. Causa-Raiz: Estoque Antigo e Fornecedores Caros (Métricas 2 e 4)

A análise da Métrica 2, combinada com a Métrica 4, revelou *por que* o custo está tão alto:

* **Variação de Fornecedor:** A Métrica 2 (detalhada) mostrou que o mesmo produto (ex: "Disco de Freio") tem custos de compra drasticamente diferentes dependendo do fornecedor. A empresa compra de fornecedores caros (ex: Custo de R$ 7.700) e depois vende pelo preço de mercado, gerando um prejuízo imenso.

* **Estoque Parado:** A Métrica 4 (Estoque) mostrou que as compras pararam em 2023, mas as vendas continuam em 2024 (Métrica 3). Isso significa que a empresa está "presa" com um estoque caro e obsoleto de 2023, forçando a vendê-lo com prejuízo em 2024 para desovar.

### 3. Ações Imediatas (Plano de Recuperação)

1.  **Revisão de Precificação (Urgente):** Parar de "vender no escuro". A empresa deve usar a Métrica 2 como base para reajustar **imediatamente** o preço de venda (`VALOR ITEM`) de todos os produtos para que seja, no mínimo, superior ao `CUSTO UNITARIO`.
2.  **Auditoria de Fornecedores:** Usar a Métrica 4 (Ranking por Valor) e a Métrica 2 (Detalhada) para identificar e **suspender compras** dos fornecedores que praticam os custos mais altos para os mesmos produtos.

### 4. Oportunidade: Clientes Fiéis (Métricas 3 e 9)

O ponto positivo é que a empresa tem uma base de clientes leais. A Métrica 3 (Perfil de Cliente) e a Métrica 9 (Lista de Compras) mostram o que eles compram:

* **Foco em "Cross-Selling":** A Métrica 9 é um mapa para venda casada. A equipe de marketing deve criar "kits" (ex: "Pastilha + Disco de Freio") e o time de vendas deve usar a Métrica 3 para ofertar produtos complementares aos clientes fiéis, aumentando o faturamento sem depender de produtos que dão prejuízo.